In [ ]:
import os
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pv.set_jupyter_backend('static')
pv.start_xvfb()
pv.set_plot_theme("document")

In [ ]:
class Poiseuille2DAnalyticalSolution:
    
    def __init__(self, L, mu, rho, ax, order):
        self.L = L
        self.mu = mu
        self.rho = rho
        self.nu = mu/rho
        self.ax = ax
        self.order = order
        pass
    
    def getU(self, t, n_nodes):
        y = np.linspace(0, self.L, n_nodes)
        u = np.zeros(n_nodes) + self.ax/2/self.nu * y * (self.L - y)
        for n in range(self.order):
            u -= 4*self.ax*self.L**2 / self.nu/np.pi**3/(2*n+1)**3 * np.sin((2*n+1)*np.pi/self.L*y) * np.exp(-(2*n+1)**2*np.pi**2/self.L**2*self.nu*t)
            pass
        return y, u
        pass
    
    pass

In [ ]:
L = 0.01
mu = 1e-3
rho = 1e3
ax = 8e-5
order = 100
poiseuille_2d = Poiseuille2DAnalyticalSolution(L, mu, rho, ax, order)
ts = np.array([2, 5.5, 9.5, 16])
# ts = np.array([2, 5.5, 9.5])
analytical_n_nodes = 21
us = np.zeros((len(ts), analytical_n_nodes))
ys = np.zeros((len(ts), analytical_n_nodes))
for i, t in enumerate(ts):
    ys[i], us[i] = poiseuille_2d.getU(t, analytical_n_nodes)
    pass

In [ ]:
file_path = "Poiseuille2DDatahalf/"
file_list = os.listdir(file_path)
file_list.sort()

def readStep(step):
    file_name = file_path + file_list[step]
    mesh = pv.read(file_name)
    return mesh
    pass

In [ ]:
mesh = readStep(1)
mesh.field_data["Time"]

In [ ]:
L = 0.01
dr = L / 80
def getUFromMeshAtTime(t, x_position, n):
    i = 0
    cur_t = 0
    while cur_t < t:
        mesh = readStep(i)
        mesh.point_data["U"] = mesh.point_data["Velocity"][:, 0]
        cur_t = mesh.field_data["Time"][0]
        i += 1
        pass
    x = np.zeros(n) + x_position
    # y = np.linspace(0, L, n)
    y = np.linspace(0, L, n)
    z = np.zeros(n)
    sample_mesh = pv.StructuredGrid(np.column_stack((x, y, z)))
    sample_mesh = sample_mesh.interpolate(mesh, radius=dr*3)
    u = sample_mesh["U"]
    return y, u
    pass

In [ ]:
x_pos = 9*L
n_sample = 81
us_sph = np.zeros((len(ts), n_sample))
ys_sph = np.zeros((len(ts), n_sample))
for i, t in enumerate(ts):
    ys_sph[i], us_sph[i] = getUFromMeshAtTime(t, x_pos, n_sample)
    pass

In [ ]:
plt.figure(figsize=(10, 5), facecolor='white')

for i, t in enumerate(ts):
    plt.plot(us_sph[i], ys_sph[i], label=f"t={t}-SPH")
    plt.scatter(us[i], ys[i], label=f"t={t}-Morris, Analytical", zorder=10, s=20, alpha=0.7)
    pass

# legend outside
plt.legend(loc= 'upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=4)
plt.grid()
plt.show()

In [ ]:
# mesh = readStep(213)
latest_file_name = file_list[-1]
mesh = pv.read(file_path + latest_file_name)
mesh.point_data["U"] = mesh.point_data["Velocity"][:, 0]
mesh.point_data["V"] = mesh.point_data["Velocity"][:, 1]
mesh

# Poiseuille 2D

assumptions:

- $\frac{\partial }{\partial x} = 0$, each plate along x-axis is the same
- $v = 0$
- when steady, $\frac{\partial }{\partial t} = 0$
- $a$ is along $x$-axis (also can be seen as a constant pressure gradient along $x$-axis $dp/dx$)
- $u(0) = u(L) = 0$

Thus the equation is reduced to:
$$
\frac{\mathrm{d}^2 u}{\mathrm{d} y^2} + \frac{a}{\nu} = 0
$$

The solution is:
$$
u(y) = \frac{a}{2\nu}y(L-y)
$$

thus $U_{\max} = \frac{a}{8\nu}L^2$

In [ ]:
# L = 0.01
# n_nodes = 51
# x = np.zeros(n_nodes) + L * 8
# y = np.linspace(0, L, n_nodes)
# z = np.zeros(n_nodes)
# structured_mesh = pv.StructuredGrid(np.column_stack((x, y, z)))
# sampled_points = structured_mesh.interpolate(mesh, radius = L/20*3)
# sampled_points

In [ ]:
# plt.figure()
# plt.plot(sampled_points.point_data["U"], y, color="k", lw=2)
# plt.grid()
# plt.show()